In [3]:
import cv2
import csv
import os
import matplotlib.pyplot as plt

def find_red_dots(image):
    # Convert the image to the RGB color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for red color in HSV
    lower_red = (0, 200, 200)
    upper_red = (0, 255, 255)

    # Create a mask to isolate red color
    red_mask = cv2.inRange(hsv_image, lower_red, upper_red)

    # Find contours in the red mask
    contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    image_height, image_width, _ = image.shape  # Get the height and width of the image


    # Extract coordinates of red dots
    red_dot_coordinates = []
    for contour in contours:
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            # Normalize the coordinates
            normalized_cX = cX / image_width
            normalized_cY = cY / image_height
            normalized_cX = normalized_cX*3000
            normalized_cY = normalized_cY*2000
            red_dot_coordinates.append((normalized_cX, normalized_cY))

    return red_dot_coordinates

def save_coordinates_to_csv(coordinates, csv_filename):
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['X', 'Y']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for coord in coordinates:
            writer.writerow({'X': coord[0], 'Y': coord[1]})

def process_images(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Loop through each file in the input folder
    for filename in os.listdir(input_folder):
        # Check if the file is an image (you can add more file format checks if needed)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f'{os.path.splitext(filename)[0]}_coordinates.csv')

            # Read the image
            image = cv2.imread(input_path)

            # Find red dots in the image
            red_dot_coordinates = find_red_dots(image)

            # Save coordinates to CSV file
            save_coordinates_to_csv(red_dot_coordinates, output_path)

if __name__ == "__main__":
    input_folder = 'D:/MSEE-2K23-AIAS/Semester 1/Machine Learning/Semester Project/UNET Model IP/Custom_Dataset/Fixation_Images/'
    output_folder = 'D:/MSEE-2K23-AIAS/Semester 1/Machine Learning/Semester Project/UNET Model IP/Custom_Dataset/Fixation_CSV/'

    process_images(input_folder, output_folder)
